# 자연어처리 + 분류
### 자연어 기반 기후기술분류 AI 경진대회 데이터 사용

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
from tqdm import tqdm

from konlpy.tag import Okt

import sklearn
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import log_loss, accuracy_score,f1_score
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from transformers import BertTokenizer, BertModel

#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
data_path = 'data/open/'

train=pd.read_csv(data_path + 'train.csv')
test=pd.read_csv(data_path +'test.csv')
sample_submission=pd.read_csv(data_path +'sample_submission.csv')

In [3]:
train=train[['과제명', '요약문_연구내용','label']]
test=test[['과제명', '요약문_연구내용']]

train['요약문_연구내용'].fillna('NAN', inplace=True)
test['요약문_연구내용'].fillna('NAN', inplace=True)

In [4]:
train['data']=train['과제명']+train['요약문_연구내용']
test['data']=test['과제명']+test['요약문_연구내용']

In [5]:
df = train.copy()

df1 = df[:100000]
df2 = df[100000:]
df1

,과제명,요약문_연구내용,label,data
0,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,(가) 외래 및 돌발해충의 발생조사 및 종 동정\n\n\n ○ 대상해충 : 최...,24,유전정보를 활용한 새로운 해충 분류군 동정기술 개발(가) 외래 및 돌발해충의 발생조...
1,대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,1차년도\n1) Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자...,0,대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...
2,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n- Tencel/...,0,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...
3,소화기 암 진단용 분자영상 형광프로브 개발,# 소화기 암 진단용 분자영상 형광프로브 개발\n\n\n- 국소 도포형 소화기 암 ...,0,소화기 암 진단용 분자영상 형광프로브 개발# 소화기 암 진단용 분자영상 형광프로브 ...
4,위암환자의 항암제반응예측을 위한 BRCA 발현검사,-In situ hybridization 검사의 정확성을 확인하기 위해 위암세포주 ...,0,위암환자의 항암제반응예측을 위한 BRCA 발현검사-In situ hybridizat...
...,...,...,...,...
99995,"재배안정성, 복합내병성 토마토 품종육성","- 토마토 유전자원의 탐색, 수집, 평가하여 육종소재 활용\n\n\n - 양친으로...",23,"재배안정성, 복합내병성 토마토 품종육성- 토마토 유전자원의 탐색, 수집, 평가하여 ..."
99996,가상현실 바람효과를 재현하는 에어블로워 클러스터링 기술 개발,&lt;주관기관 개발내용&gt;\n\n\n？ 가상현실 연동용 윈드블로워 개발\n\n...,0,가상현실 바람효과를 재현하는 에어블로워 클러스터링 기술 개발&lt;주관기관 개발내용...
99997,혈관재협착을 일으키는 평활근전구세포의 REDOX 반응 연구,1차년도 혈관전구세포의 분화에 미치는 redox 단백질의 기능 연구\n\n1) iP...,0,혈관재협착을 일으키는 평활근전구세포의 REDOX 반응 연구1차년도 혈관전구세포의 분...
99998,갈색날개매미충의 신호화학물질 개발 및 이용방법 연구,○ 외래해충 유인 및 기피물질 선발\n\n\n ○ 선발된 유인물질과 기피물질의 작용...,24,갈색날개매미충의 신호화학물질 개발 및 이용방법 연구○ 외래해충 유인 및 기피물질 선...


In [6]:
okt = Okt()

def text_preprocess(x):
    text=[]
    a = re.sub('[^가-힣0-9a-zA-Z\\s]', '',x)
    for j in a.split():
        text.append(j)
    return ' '.join(text)

def tokenize(x):
    text = []
    tokens = okt.pos(x)
    for token in tokens :
        if token[1] == 'Adjective' or token[1]=='Adverb' or token[1] == 'Determiner' or token[1] == 'Noun' or token[1] == 'Verb' or 'Unknown':
            text.append(token[0])
    return text

In [7]:
tqdm.pandas()
df1['contents'] = df1['data'].apply(lambda x : text_preprocess(x))
df1['contents'] = df1['contents'].progress_apply(lambda x: tokenize(x))
df1

c:\users\samsung\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [1:40:42<00:00, 16.55it/s]
c:\users\samsung\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,과제명,요약문_연구내용,label,data,contents
0,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,(가) 외래 및 돌발해충의 발생조사 및 종 동정\n\n\n ○ 대상해충 : 최...,24,유전정보를 활용한 새로운 해충 분류군 동정기술 개발(가) 외래 및 돌발해충의 발생조...,"[유전, 정보, 를, 활용, 한, 새로운, 해충, 분류군, 동정, 기술, 개발, 가..."
1,대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,1차년도\n1) Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자...,0,대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,"[대장암, 의, TRAIL, 내, 성, 표적, 인자, 발굴, 및, TRAIL, 반응..."
2,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n- Tencel/...,0,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,"[비목, 질계, 셀룰로오스, 식물, 자원, 을, 활용, 한, 기능, 성, 부직포, ..."
3,소화기 암 진단용 분자영상 형광프로브 개발,# 소화기 암 진단용 분자영상 형광프로브 개발\n\n\n- 국소 도포형 소화기 암 ...,0,소화기 암 진단용 분자영상 형광프로브 개발# 소화기 암 진단용 분자영상 형광프로브 ...,"[소화기, 암, 진단, 용, 분자영상, 형광, 프로, 브, 개발, 소화기, 암, 진..."
4,위암환자의 항암제반응예측을 위한 BRCA 발현검사,-In situ hybridization 검사의 정확성을 확인하기 위해 위암세포주 ...,0,위암환자의 항암제반응예측을 위한 BRCA 발현검사-In situ hybridizat...,"[위암, 환자, 의, 항암제, 반응, 예측, 을, 위, 한, BRCA, 발현, 검사..."
...,...,...,...,...,...
99995,"재배안정성, 복합내병성 토마토 품종육성","- 토마토 유전자원의 탐색, 수집, 평가하여 육종소재 활용\n\n\n - 양친으로...",23,"재배안정성, 복합내병성 토마토 품종육성- 토마토 유전자원의 탐색, 수집, 평가하여 ...","[재배, 안정, 성, 복합, 내, 병, 성, 토마토, 품종, 육성, 토마토, 유전자..."
99996,가상현실 바람효과를 재현하는 에어블로워 클러스터링 기술 개발,&lt;주관기관 개발내용&gt;\n\n\n？ 가상현실 연동용 윈드블로워 개발\n\n...,0,가상현실 바람효과를 재현하는 에어블로워 클러스터링 기술 개발&lt;주관기관 개발내용...,"[가상현실, 바람, 효과, 를, 재현, 하는, 에어, 블, 로, 워, 클러스터링, ..."
99997,혈관재협착을 일으키는 평활근전구세포의 REDOX 반응 연구,1차년도 혈관전구세포의 분화에 미치는 redox 단백질의 기능 연구\n\n1) iP...,0,혈관재협착을 일으키는 평활근전구세포의 REDOX 반응 연구1차년도 혈관전구세포의 분...,"[혈관, 재협착, 을, 일으키는, 평활, 근, 전, 구, 세포, 의, REDOX, ..."
99998,갈색날개매미충의 신호화학물질 개발 및 이용방법 연구,○ 외래해충 유인 및 기피물질 선발\n\n\n ○ 선발된 유인물질과 기피물질의 작용...,24,갈색날개매미충의 신호화학물질 개발 및 이용방법 연구○ 외래해충 유인 및 기피물질 선...,"[갈색, 날개, 매미, 충의, 신호, 화학물질, 개발, 및, 이용, 방법, 연구, ..."


In [8]:
df1.to_csv("df1.csv", index=False, encoding="utf-8-sig")

In [ ]:
tqdm.pandas()
df1['contents'] = df1['data'].apply(lambda x : text_preprocess(x))
df1['contents'] = df1['contents'].progress_apply(lambda x: tokenize(x))
df1

In [13]:
import gensim

sentences = df['contents'].values
model = gensim.models.Word2Vec(sentences)

In [14]:
words = list(model.wv.vocab)
filename = '벡터.txt'
model.wv.save_word2vec_format(filename, binary=False)

In [15]:
import os

embeddings_index = {}
f = open(os.path.join('', '벡터.txt'),  encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

In [16]:
num = int(len(df) *0.8)

In [52]:
X_train = df.loc[:num, 'contents'].values
y_train = df.loc[:num, 'label'].values
X_test = df.loc[num:, 'contents'].values
y_test = df.loc[num:, 'label'].values

total_reviews = df.values
max_length = 200
review_lines = df['contents'].tolist()

In [53]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

VALIDATION_SPLIT = 0.1

# vectorize the text samples into a 2D integer tensor
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(review_lines)
sequences = tokenizer_obj.texts_to_sequences(review_lines)

# pad sequences
word_index = tokenizer_obj.word_index
print('Found %s unique tokens.' % len(word_index))

review_pad = pad_sequences(sequences, maxlen=max_length)
rating =  df['label'].values
print('Shape of review tensor:', review_pad.shape)
print('Shape of sentiment tensor:', rating.shape)

# split the data into a training set and a validation set
indices = np.arange(review_pad.shape[0])
np.random.shuffle(indices)
review_pad = review_pad[indices]
sentiment = rating[indices]
num_validation_samples = int(VALIDATION_SPLIT * review_pad.shape[0])

X_train_pad = review_pad[:-num_validation_samples]
y_train = sentiment[:-num_validation_samples]
X_test_pad = review_pad[-num_validation_samples:]
y_test = sentiment[-num_validation_samples:]

Found 58962 unique tokens.
Shape of review tensor: (10000, 200)
Shape of sentiment tensor: (10000,)


In [20]:
EMBEDDING_DIM = 100
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print(num_words)

58963


In [33]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Embedding, Flatten, Dropout
from tensorflow.python.keras.layers.convolutional import Conv1D
from tensorflow.python.keras.layers.convolutional import MaxPooling1D
from tensorflow.python.keras.initializers import Constant


model = Sequential()
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_length,
                            trainable=False)

model.add(embedding_layer)

model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))

model.add(MaxPooling1D(pool_size=2))

model.add(Dense(128, activation='relu'))

model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(46, activation='sigmoid'))


print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 100)          5896300   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 196, 128)          64128     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 98, 128)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 98, 128)           16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 98, 128)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 46)               

In [67]:
Y = np.concatenate([y_train,y_test])
Y
len(Y)

10000

In [68]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [69]:
len(dummy_y)

10000

In [71]:
import datetime
stime2 = datetime.datetime.now()
print("training start")

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


hist= model.fit(X_train_pad, dummy_y[:9000], batch_size=64, epochs=5, 
                validation_data=(X_test_pad, dummy_y[9000:]), verbose=2)

etime2 = datetime.datetime.now()
mtime2 = etime2-stime2

print("training complete")
print("종료시간: ",etime2,"\n소요시간: ",mtime2,'\n\n')

loss, accuracy = model.evaluate(X_test_pad, dummy_y[9000:], batch_size=64)
print('Accuracy: %.2f' % (accuracy*100),'%')

training start
Train on 9000 samples, validate on 1000 samples
Epoch 1/5
9000/9000 - 11s - loss: 0.0767 - accuracy: 0.9773 - val_loss: 1.4459 - val_accuracy: 0.8080
Epoch 2/5
9000/9000 - 11s - loss: 0.0418 - accuracy: 0.9871 - val_loss: 1.4648 - val_accuracy: 0.8080
Epoch 3/5
9000/9000 - 11s - loss: 0.0338 - accuracy: 0.9901 - val_loss: 1.5435 - val_accuracy: 0.8120
Epoch 4/5
9000/9000 - 11s - loss: 0.0288 - accuracy: 0.9919 - val_loss: 1.9329 - val_accuracy: 0.8140
Epoch 5/5
9000/9000 - 11s - loss: nan - accuracy: 0.9203 - val_loss: nan - val_accuracy: 0.8090
training complete
종료시간:  2021-07-05 21:56:01.310722 
소요시간:  0:00:55.250762 


1000/1000 [==============================] - 0s 417us/sample - loss: nan - accuracy: 0.8090
Accuracy: 80.90 %


In [48]:
len(df['label'].unique())

46

# --------------------------------------------------------
## BERT 모델: 메모리 부족으로 실행불가

In [8]:
new_train = list(zip(train.data, train.label))

In [39]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [42]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F

In [45]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0


model.train()
for epoch in range(epochs):
    
    for text, label in new_train:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        print(itr)
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:79] data. DefaultCPUAllocator: not enough memory: you tried to allocate 2642411520 bytes.

# ------------------------------------------------------------------------------------------------

In [ ]:
results = cls_model.predict(test_inputs)
results=tf.argmax(results, axis=1)

In [ ]:
sample_submission['label']=results

In [ ]:
sample_submission

In [ ]:
sample_submission.to_csv('bert_baseline.csv', index=False)